# Elementos escenciales

Alberga la base del sistema y la conexión.

## Librerías

Importamos las librerías necesarias para el funcionamiento del código.

In [1]:
import oracledb
import os 
from pathlib import Path
from datetime import datetime
from typing import Optional

## Conexión reutilizable

Se crea una función para conectarse a la base de datos local.

In [2]:
def get_connection():
    return oracledb.connect(
        user="SYSTEM",
        password="96443804",
        dsn="localhost:1521/xe"
    ) 

# Creación de la base de datos

Código necesario para crear las tablas que representan las entidades de nuestro modelo.

## Esquema base de datos

Se define una función para crear todas las tablas a partir de una serie de consultas.

In [3]:
def create_schema(query):
    try:         
        with get_connection() as conn:             
            with conn.cursor() as cur:                 
                cur.execute(query)                 
                print(f'✓ Tabla creada exitosamente')
            conn.commit()     
    except oracledb.DatabaseError as e:        
        err = e
        if 'ORA-00955' in str(err):  # Tabla ya existe
            pass  # No imprimir nada si la tabla ya existe
        else:
            print(f"Error al crear tabla: {err}")

## Tablas de la base de datos

Se crea una función que alberga las consultas necesarias para la creación de las tablas y que llama la función para crear las tablas.

In [ ]:
def create_all_tables():
    tables = [
        (
             "CREATE TABLE "
             "PERSONAS ("
             "RUT VARCHAR(10) PRIMARY KEY,"
             "NOMBRES VARCHAR(64),"
             "APELLIDOS VARCHAR(64))"
        ),
        (
             "CREATE TABLE "
             "ASIGNATURAS ("
             "ID INTEGER PRIMARY KEY,"
             "NOMBRE VARCHAR(32))"
        ),
        (
             "CREATE TABLE "
             "PROFESORES ("
             "ID INTEGER PRIMARY KEY,"
             "PROFESOR VARCHAR(10),"
             "ASIGNATURA INTEGER,"
             "FOREIGN KEY (PROFESOR) REFERENCES PERSONAS(RUT),"
             "FOREIGN KEY (ASIGNATURA) REFERENCES ASIGNATURA(ID))"
        ),
        (
            "CREATE TABLE "
            "CURSOS ("
            "ID VARCHAR(10) PRIMARY KEY,"
            "ALUMNO VARCHAR(10),"
            "FOREIGN KEY (ALUMNO) REFERENCES PERSONAS(RUT))"
        ),
        (
            "CREATE TABLE "
            "HORARIOS ("
            "ID INTEGER PRIMARY KEY,"
            "PROFESOR INTEGER,"
            "CURSO VARCHAR(10),"
            "FOREIGN KEY (PROFESOR) REFERENCES PROFESORES(ID),"
            "FOREIGN KEY (CURSO) REFERENCES CURSOS(ID))"
        ),
        (
            "CREATE TABLE "
            "EVALUACIONES ("
            "ID INTEGER PRIMARY KEY,"
            "NOTA INTEGER,"
            "ASIGNATURA INTEGER,"
            "ALUMNO VARCHAR(10),"
            "PROFESOR INTEGER,"
            "FOREIGN KEY (ASIGNATURA) REFERENCES ASIGNATURA(ID),"
            "FOREIGN KEY (ALUMNO) REFERENCES PERSONAS(RUT),"
            "FOREIGN KEY (PROFESOR) REFERENCES PROFESORES(ID))"
        )
    ]
    for query in tables:
        create_schema(query)


## Script para limpiar tablas previamente creadas

Para agilizar el proceso de testeo y demostración, se creó una función que limpiará la base de datos si es que esta estaba previamente poblada.

In [ ]:
try:
    conn = oracledb.connect(get_connection())
    cur = conn.cursor()
    
    # Orden de eliminación (respetando foreign keys)
    tablas = ['EVALUACIONES', 'HORARIOS', 'CURSOS', 'PROFESORES', 'ASIGNATURAS', 'PERSONAS']
    
    for tabla in tablas:
        try:
            cur.execute(f'DROP TABLE {tabla}')
            conn.commit()
            print(f'✓ Tabla {tabla} eliminada')
        except oracledb.DatabaseError as e:
            if 'ORA-00942' in str(e):  # Tabla no existe
                print(f'- Tabla {tabla} no existe (ignorado)')
            else:
                print(f'✗ Error al eliminar {tabla}: {e}')
    
    conn.close()
    print("\n✓ Limpieza completada exitosamente")
    
except oracledb.DatabaseError as e:
    print(f"Error de conexión: {e}")

✓ Tabla EVALUACIONES eliminada
✓ Tabla HORARIOS eliminada
✓ Tabla CURSOS eliminada
✓ Tabla PROFESORES eliminada
✓ Tabla ASIGNATURA eliminada
✓ Tabla PERSONAS eliminada

✓ Limpieza completada exitosamente
✓ Tabla PROFESORES eliminada
✓ Tabla ASIGNATURA eliminada
✓ Tabla PERSONAS eliminada

✓ Limpieza completada exitosamente


## Creación de la base de datos

Se finaliza la creación de la base llamando a la función necesaria.

In [16]:
create_all_tables()

✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente


# Funciones para insertar datos

Debido a que la estructura de cada tabla varía, es necesario crear funciones específicas para insertar datos en la entidad correspondiente.

## Tabla Personas

In [7]:
def create_persona(rut=None, nombres=None, apellidos=None):     
    sql = (        
    "INSERT INTO personas (rut, nombres, apellidos) "         
    "VALUES (:rut, :nombres, :apellidos)"     
    )     
    try:
        with get_connection() as conn:         
            with conn.cursor() as cur:             
                cur.execute(sql, {                 
                "rut": rut,                 
                "nombres": nombres,                 
                "apellidos": apellidos             
                })             
            conn.commit()            
        print(f"✓ Persona con RUT={rut} creada.")
    except oracledb.DatabaseError as e:
        print(f"Error al crear persona: {e}") 

## Tabla Asignaturas

In [ ]:
def create_asignatura(id=None, nombre=None):     
    sql = (        
    "INSERT INTO ASIGNATURAS (id, nombre)"         
    "VALUES (:id, :nombre)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "id": id,                 
            "nombre": nombre          
            })             
        conn.commit()            
    print(f"Departamento {nombre} creado.") 

## Tabla Profesores

In [ ]:
def create_profesor(ID=None, PROFESOR=None, ASIGNATURA=None):     
    sql = (        
    "INSERT INTO PROFESORES (ID, PROFESOR, ASIGNATURA)"         
    "VALUES (:ID, :PROFESOR, :ASIGNATURA)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,
            "PROFESOR": PROFESOR,                 
            "ASIGNATURA": ASIGNATURA        
            })             
        conn.commit()            
    print(f"Profesor {PROFESOR} registrado.") 

## Tabla Cursos

In [ ]:
def create_curso(ID=None, ALUMNO=None):     
    sql = (        
    "INSERT INTO CURSOS (ID, ALUMNO)"         
    "VALUES (:ID, :ALUMNO)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,                 
            "ALUMNO": ALUMNO        
            })             
        conn.commit()            
    print(f"Alumno {ALUMNO} registrado en curso {ID}.")     


## Tabla Horarios

In [ ]:
def create_horario(ID=None, PROFESOR=None, CURSO=None):     
    sql = (        
    "INSERT INTO HORARIOS (ID, PROFESOR, CURSO)"         
    "VALUES (:ID, :PROFESOR, :CURSO)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,                 
            "PROFESOR": PROFESOR,
            "CURSO": CURSO    
            })             
        conn.commit()            
    print(f"Profesor {PROFESOR} dictará curso {CURSO}.")    


## Tabla Evaluaciones

In [ ]:
def create_evaluacion(ID=None, NOTA=None, ASIGNATURA=None, ALUMNO=None, PROFESOR=None):     
    sql = (        
    "INSERT INTO EVALUACIONES (ID, NOTA, ASIGNATURA, ALUMNO, PROFESOR)"         
    "VALUES (:ID, :NOTA, :ASIGNATURA, :ALUMNO, :PROFESOR)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,
            "NOTA": NOTA,
            "ASIGNATURA": ASIGNATURA,
            "ALUMNO": ALUMNO,                 
            "PROFESOR": PROFESOR 
            })             
        conn.commit()            
    print(f"Alumno {ALUMNO} recibió la nota {NOTA}.") 


## Poblamiento de tablas

### Poblar tabla Personas

In [ ]:
create_persona("12345678-9", "Juan", "Pérez")
create_persona("23456789-0", "María", "González")
create_persona("34567890-1", "Carlos", "López")
create_persona("45678901-K", "Ana", "Martínez")
create_persona("56789012-3", "Pedro", "Sánchez")
create_persona("67890123-4", "Laura", "Rodríguez")

### Poblar tabla Asignaturas

In [ ]:
create_asignatura(1, "Matemáticas")
create_asignatura(2, "Lenguaje")
create_asignatura(3, "Historia")
create_asignatura(4, "Ciencias")
create_asignatura(5, "Educación Física")

### Poblar tabla Profesores

In [ ]:
create_profesor(1, "12345678-9", 1)
create_profesor(2, "23456789-0", 2)
create_profesor(3, "34567890-1", 3)
create_profesor(4, "45678901-K", 4)
create_profesor(5, "56789012-3", 5)

### Poblar tabla Cursos

In [ ]:
create_curso("3-B", "67890123-4")
create_curso("6-A", "56789012-3")
create_curso("5-B", "45678901-K")
create_curso("7-A", "34567890-1")
create_curso("IV-A", "23456789-0")

### Poblar tabla Horarios

In [ ]:
create_horario(1, 1, "3-B")
create_horario(2, 2, "6-A")
create_horario(3, 3, "5-B")
create_horario(4, 4, "7-A")
create_horario(5, 5, "IV-A")

### Poblar tabla Evaluaciones

In [ ]:
create_evaluacion(1, 95, 1, "67890123-4", 1)
create_evaluacion(2, 87, 2, "56789012-3", 2)
create_evaluacion(3, 92, 3, "45678901-K", 3)
create_evaluacion(4, 78, 4, "34567890-1", 4)
create_evaluacion(5, 88, 5, "23456789-0", 5)

# Funciones para leer datos

## Leer todos los datos de una tabla

Se definen funciones para obtener todos los datos de una tabla determinada o para buscar un elemento de una tabla determinada.

In [13]:
def read_table(tabla):
    sql = (
        f"SELECT * FROM {tabla}"
    )
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                resultados = cur.execute(sql)
                print(f"Consulta a la tabla {tabla}")
                for fila in resultados:
                    print(fila)
    except oracledb.DatabaseError as e:
        err = e
        print(f"Error al leer datos: {err}")

## Buscar por identificador

In [14]:
def read_table_by_id(tabla, columna_id, valor_id):
    sql = (
        f"SELECT * FROM {tabla} WHERE {columna_id} = :valor"
    )
    parametros = {"valor": valor_id}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                resultados = cur.execute(sql, parametros)
                print(f"✓ Consulta a la tabla {tabla} donde {columna_id} = {valor_id}")
                encontrados = 0
                for fila in resultados:
                    print(f"  {fila}")
                    encontrados += 1
                if encontrados == 0:
                    print(f"  No se encontraron registros")
    except oracledb.DatabaseError as e:
        print(f"Error al leer datos: {e}")

# Funciones para actualizar datos

## Actualizar Persona

In [ ]:
def update_persona(
    RUT,
    NOMBRES: Optional[str] = None,
    APELLIDOS: Optional[str] = None
):
    modificaciones = []
    parametros = {"RUT": RUT}

    if RUT is not None:
        modificaciones.append("RUT =: RUT")
        parametros["RUT"] = RUT
    if NOMBRES is not None:
        modificaciones.append("NOMBRES =: NOMBRES")
        parametros["NOMBRES"] = NOMBRES
    if APELLIDOS is not None:
        modificaciones.append("APELLIDOS =: APELLIDOS")
        parametros["APELLIDOS"] = APELLIDOS
    if not modificaciones:
        return print("No hay campos para actualizar.")

    sql = f"UPDATE PERSONAS SET {", ".join(modificaciones)} WHERE RUT =: RUT"

    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
        print(f"Persona con RUT={RUT} actualizada.")

## Actualizar Asignatura

In [ ]:
def update_asignatura(
    ID: int,
    NOMBRE: Optional[str] = None
):
    modificaciones = []
    parametros = {"ID": ID}

    if NOMBRE is not None:
        modificaciones.append("NOMBRE = :NOMBRE")
        parametros["NOMBRE"] = NOMBRE

    if not modificaciones:
        print("No hay campos para actualizar en Asignaturas.")
        return

    sql = f"UPDATE ASIGNATURAS SET {', '.join(modificaciones)} WHERE ID = :ID"
    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
    print(f"Asignatura ID={ID} actualizada.")

## Actualizar Profesor

In [ ]:
def update_profesor(
    ID: int,
    PROFESOR: Optional[str] = None,
    ASIGNATURA: Optional[int] = None
):
    modificaciones = []
    parametros = {"ID": ID}

    if PROFESOR is not None:
        modificaciones.append("PROFESOR = :PROFESOR")
        parametros["PROFESOR"] = PROFESOR
    if ASIGNATURA is not None:
        modificaciones.append("ASIGNATURA = :ASIGNATURA")
        parametros["ASIGNATURA"] = ASIGNATURA

    if not modificaciones:
        print("No hay campos para actualizar en Profesores.")
        return

    sql = f"UPDATE PROFESORES SET {', '.join(modificaciones)} WHERE ID = :ID"
    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
    print(f"Profesor ID={ID} actualizado.")

## Actualizar Curso

In [ ]:
def update_curso(
    ID: str,
    ALUMNO: Optional[str] = None
):
    modificaciones = []
    parametros = {"ID": ID}

    if ALUMNO is not None:
        modificaciones.append("ALUMNO = :ALUMNO")
        parametros["ALUMNO"] = ALUMNO

    if not modificaciones:
        print("No hay campos para actualizar en Cursos.")
        return

    sql = f"UPDATE CURSOS SET {', '.join(modificaciones)} WHERE ID = :ID"
    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
    print(f"Curso ID={ID} actualizado.")

## Actualizar Horario

In [ ]:
def update_horario(
    ID: int,
    PROFESOR: Optional[int] = None,
    CURSO: Optional[str] = None
):
    modificaciones = []
    parametros = {"ID": ID}

    if PROFESOR is not None:
        modificaciones.append("PROFESOR = :PROFESOR")
        parametros["PROFESOR"] = PROFESOR
    if CURSO is not None:
        modificaciones.append("CURSO = :CURSO")
        parametros["CURSO"] = CURSO

    if not modificaciones:
        print("No hay campos para actualizar en Horarios.")
        return

    sql = f"UPDATE HORARIOS SET {', '.join(modificaciones)} WHERE ID = :ID"
    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
    print(f"Horario ID={ID} actualizado.")

## Actualizar Evaluación

In [ ]:
def update_evaluacion(
    ID: int,
    NOTA: Optional[int] = None,
    ASIGNATURA: Optional[int] = None,
    ALUMNO: Optional[str] = None,
    PROFESOR: Optional[int] = None
):
    modificaciones = []
    parametros = {"ID": ID}

    if NOTA is not None:
        modificaciones.append("NOTA = :NOTA")
        parametros["NOTA"] = NOTA
    if ASIGNATURA is not None:
        modificaciones.append("ASIGNATURA = :ASIGNATURA")
        parametros["ASIGNATURA"] = ASIGNATURA
    if ALUMNO is not None:
        modificaciones.append("ALUMNO = :ALUMNO")
        parametros["ALUMNO"] = ALUMNO
    if PROFESOR is not None:
        modificaciones.append("PROFESOR = :PROFESOR")
        parametros["PROFESOR"] = PROFESOR

    if not modificaciones:
        print("No hay campos para actualizar en Evaluaciones.")
        return

    sql = f"UPDATE EVALUACIONES SET {', '.join(modificaciones)} WHERE ID = :ID"
    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
    print(f"Evaluación ID={ID} actualizada.")

# Funciones para eliminar datos

## Eliminar Persona

In [ ]:
def delete_persona(RUT):
    sql = (
        "DELETE FROM PERSONAS WHERE RUT = :RUT"
    )
    parametros = {"RUT" : RUT}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
            print(f"Persona con RUT {RUT} eliminada.")
    except oracledb.DatabaseError as e:
        err = e
        print(f"Error al eliminar dato: {err} \n {sql} \n {parametros}")

## Eliminar Asignatura

In [ ]:
def delete_asignatura(ID: int):
    sql = "DELETE FROM ASIGNATURAS WHERE ID = :ID"
    parametros = {"ID": ID}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
        print(f"Asignatura ID={ID} eliminada.")
    except oracledb.DatabaseError as e:
        print(f"Error al eliminar asignatura ID={ID}: {e}")

## Eliminar Profesor

In [ ]:
def delete_profesor(ID: int):
    sql = "DELETE FROM PROFESORES WHERE ID = :ID"
    parametros = {"ID": ID}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
        print(f"Profesor ID={ID} eliminado.")
    except oracledb.DatabaseError as e:
        print(f"Error al eliminar profesor ID={ID}: {e}")

## Eliminar Curso

In [ ]:
def delete_curso(ID: str):
    sql = "DELETE FROM CURSOS WHERE ID = :ID"
    parametros = {"ID": ID}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
        print(f"Curso ID={ID} eliminado.")
    except oracledb.DatabaseError as e:
        print(f"Error al eliminar curso ID={ID}: {e}")

## Eliminar Horario

In [ ]:
def delete_horario(ID: int):
    sql = "DELETE FROM HORARIOS WHERE ID = :ID"
    parametros = {"ID": ID}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
        print(f"Horario ID={ID} eliminado.")
    except oracledb.DatabaseError as e:
        print(f"Error al eliminar horario ID={ID}: {e}")

## Eliminar Evaluación

In [ ]:
def delete_evaluacion(ID: int):
    sql = "DELETE FROM EVALUACIONES WHERE ID = :ID"
    parametros = {"ID": ID}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
        print(f"Evaluación ID={ID} eliminada.")
    except oracledb.DatabaseError as e:
        print(f"Error al eliminar evaluación ID={ID}: {e}")